In this tutorial, we show how to use RipsNet to learn the topology of very simple synthetic data sets comprised of (noisy) circles. Let's first define the seed (for reproducibility), the number of point clouds that we will use for training RipsNet, the number of point clouds we will use to test it, the cardinality of the point clouds and finally the amount of corrupted points (when noise is added).

In [9]:
N_sets_train = 1800  # Number of train point clouds
N_sets_test  = 300  # Number of test  point clouds
N_points     = 600  # Point cloud cardinality
N_noise      = 200  # Number of corrupted points

### Imports


In [10]:
!pip install gudhi tqdm matplotlib scikit-learn scipy torch xgboost

/bin/bash: line 1: pip: command not found


In [11]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import gudhi as gd
from gudhi.representations import DiagramSelector, Landscape, PersistenceImage
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adamax
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'gudhi'

###utils


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adamax

class DenseRagged(nn.Module):
    def __init__(self, in_features=None, out_features=30, activation='relu', use_bias=True):
        super(DenseRagged, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.use_bias = use_bias
        self.activation = activation

        self.weight = None
        self.bias = None

    def forward(self, inputs):
        outputs = []
        for x in inputs:
            if self.weight is None:
                in_features = x.shape[-1]
                self.weight = nn.Parameter(torch.randn(in_features, self.out_features) * 0.01)
                if self.use_bias:
                    self.bias = nn.Parameter(torch.zeros(self.out_features))
            y = torch.matmul(x, self.weight)
            if self.use_bias:
                y = y + self.bias
            if self.activation == 'relu':
                y = F.relu(y)
            elif self.activation == 'sigmoid':
                y = torch.sigmoid(y)
            elif self.activation == 'tanh':
                y = torch.tanh(y)
            outputs.append(y)
        return outputs


class PermopRagged(nn.Module):
    def forward(self, inputs):
        return torch.stack([torch.sum(x, dim=0) for x in inputs])


class RaggedPersistenceModel(nn.Module):
    def __init__(self, output_dim):
        super(RaggedPersistenceModel, self).__init__()
        self.ragged_layers = nn.ModuleList([
            DenseRagged(out_features=30, activation='relu'),
            DenseRagged(out_features=20, activation='relu'),
            DenseRagged(out_features=10, activation='relu')
        ])
        self.perm = PermopRagged()

        self.fc = nn.Sequential(
            nn.Linear(10, 50),
            nn.ReLU(),
            nn.Linear(50, 100),
            nn.ReLU(),
            nn.Linear(100, 200),
            nn.ReLU(),
            nn.Linear(200, output_dim),
            nn.Sigmoid()
        )

    def forward(self, inputs):
        x = inputs
        for layer in self.ragged_layers:
            x = layer(x)
        x = self.perm(x)
        x = self.fc(x)
        return x

import numpy as np
from tqdm import tqdm
import gudhi as gd


####################################
###  Creation of point clouds    ###
####################################

def create_circle(N_points, r, x_0, y_0):
    X = []
    for i in range(N_points):
        theta = np.random.uniform() * 2 * np.pi
        X.append([(r * np.cos(theta)) + x_0, (r * np.sin(theta) + y_0)])
    return np.array(X)


def create_1_circle_clean(N_points):
    r = 2
    x_0, y_0 = 10 * np.random.rand() - 5, 10 * np.random.rand() - 5
    return create_circle(N_points, r, x_0, y_0)


def create_2_circle_clean(N_points):
    r1 = 5
    r2 = 3
    x_0, y_0 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    x_1, y_1 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    while np.sqrt((x_0 - x_1) ** 2 + (y_0 - y_1) ** 2) <= r1 + r2:
        x_1, y_1 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    circle1 = create_circle(N_points // 2, r1, x_0, y_0)
    circle2 = create_circle(N_points - N_points // 2, r2, x_1, y_1)
    X = [0] * N_points
    X[:N_points // 2] = circle1
    X[N_points // 2:] = circle2
    np.random.shuffle(X)
    return np.array(X)


def create_3_circle_clean(N_points):
    r0 = 5
    r1 = 3
    r2 = 2
    x_0, y_0 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    x_1, y_1 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    while np.sqrt((x_0 - x_1) ** 2 + (y_0 - y_1) ** 2) <= r0 + r1:
        x_1, y_1 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15

    x_2, y_2 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    while(np.sqrt((x_0 - x_2)**2 + (y_0 - y_2)**2) <= r0 + r2) or (np.sqrt((x_1 - x_2)**2 + (y_1 - y_2)**2) <= r1 + r2):
        x_2, y_2 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15

    circle0 = create_circle(N_points // 3, r0, x_0, y_0)
    circle1 = create_circle(N_points // 3, r1, x_1, y_1)
    circle2 = create_circle(N_points // 3, r2, x_2, y_2)

    # Handler in case N_points mod 3 != 0.
    true_N_points = 3 * (N_points // 3)

    X = [[0,0]] * true_N_points
    X[:true_N_points // 3] = circle0
    X[true_N_points // 3:2 * true_N_points // 3] = circle1
    X[2 * true_N_points // 3:] = circle2
    np.random.shuffle(X)
    return np.array(X)


def create_1_circle_noisy(N_points, N_noise):
    r = 2
    x_0, y_0 = 10 * np.random.rand() - 5, 10 * np.random.rand() - 5
    X = create_circle(N_points, r, x_0, y_0)
    noise = []
    for i in range(N_noise):
        noise.append([np.random.uniform(x_0 - r, x_0 + r),
                      np.random.uniform(y_0 - r, y_0 + r)])
    X = np.array(X)
    X[np.random.choice(np.arange(len(X)), size=N_noise, replace=False, p=None)] = np.array(noise)
    return X


def create_2_circle_noisy(N_points, N_noise):
    r1 = 5
    r2 = 3
    x_0, y_0 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    x_1, y_1 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    while(np.sqrt((x_0 - x_1)**2 + (y_0 - y_1)**2) <= r1 + r2):
        x_1, y_1 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    circle1 = create_circle(N_points // 2, r1, x_0, y_0)
    circle2 = create_circle(N_points - N_points // 2, r2, x_1, y_1)
    X = [0] * N_points
    X[:N_points // 2] = circle1
    X[N_points // 2:] = circle2
    np.random.shuffle(X)
    noise = []
    for i in range(N_noise):
        noise.append([np.random.uniform(min(x_0 - r1, x_1 - r2), max(x_0 + r1, x_1 + r2)),
                      np.random.uniform(min(y_0 - r1, y_1 - r2), max(y_0 + r1, y_1 + r2))])
    X = np.array(X)
    X[np.random.choice(np.arange(len(X)), size=N_noise, replace=False, p=None)] = np.array(noise)
    return X


def create_3_circle_noisy(N_points, N_noise):
    r0 = 5
    r1 = 3
    r2 = 2
    x_0, y_0 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    x_1, y_1 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    while np.sqrt((x_0 - x_1) ** 2 + (y_0 - y_1) ** 2) <= r0 + r1:
        x_1, y_1 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    x_2, y_2 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    while(np.sqrt((x_0 - x_2)**2 + (y_0 - y_2)**2) <= r0 + r2) or (np.sqrt((x_1 - x_2)**2 + (y_1 - y_2)**2) <= r1 + r2):
        x_2, y_2 = 30 * np.random.rand() - 15, 30 * np.random.rand() - 15
    circle0 = create_circle(N_points // 3, r0, x_0, y_0)
    circle1 = create_circle(N_points // 3, r1, x_1, y_1)
    circle2 = create_circle(N_points // 3, r2, x_2, y_2)

    true_N_points = 3 * (N_points // 3)
    X = [[0,0]] * true_N_points
    X[:true_N_points // 3] = circle0
    X[true_N_points // 3:2 * true_N_points // 3] = circle1
    X[2 * true_N_points // 3:] = circle2

    np.random.shuffle(X)
    noise = []
    for i in range(N_noise):
        noise.append([np.random.uniform(np.min([x_0 - r0, x_1 - r1, x_2 - r2]), np.max([x_0 + r0, x_1 + r1, x_2 + r2])),
                      np.random.uniform(np.min([y_0 - r0, y_1 - r1, y_2 - r2]), np.max([y_0 + r0, y_1 + r1, y_2 + r2]))])
    X = np.array(X)
    X[np.random.choice(np.arange(len(X)), size=N_noise, replace=False, p=None)] = np.array(noise)
    return X

def augment_isometries(pc, n, rng, trans_frac=0.08):
    bbox = pc.max(axis=0) - pc.min(axis=0)
    t_max = trans_frac * np.linalg.norm(bbox)  # translation scale relative to cloud size
    augmented = []
    for _ in range(n):
        theta = rng.uniform(0, 2 * np.pi)
        R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
        t = rng.uniform(-t_max, t_max, size=2)
        augmented.append((pc @ R.T) + t)
    return augmented

def data_augmentation_by_isometries(data_train, label_train, n_augment_per_sample, seed=42):
    rng = np.random.default_rng(seed)
    aug_data, aug_labels = [], []
    for pc, lbl in zip(data_train, label_train):
        aug_data.append(pc)
        aug_labels.append(lbl)
        for pc_aug in augment_isometries(pc, n_augment_per_sample, rng):
            aug_data.append(pc_aug)
            aug_labels.append(lbl)
    return aug_data, aug_labels

def create_multiple_circles(N_sets_train, N_points, noisy=False, N_noise=0, n_augment_per_sample = 0):

    data_train, PD_train = [[] for _ in range(N_sets_train)], []
    label_train = np.zeros((N_sets_train,))

    if not noisy:
        for i in tqdm(range(N_sets_train // 3)):
            data_train[i] = create_1_circle_clean(N_points)
            label_train[i] = 1
        for i in tqdm(range(N_sets_train // 3, 2 * N_sets_train // 3)):
            data_train[i] = create_2_circle_clean(N_points)
            label_train[i] = 2
        for i in tqdm(range(2 * N_sets_train // 3, N_sets_train)):
            data_train[i] = create_3_circle_clean(N_points)
            label_train[i] = 3
    else:
        for i in tqdm(range(N_sets_train // 3)):
            data_train[i] = create_1_circle_noisy(N_points, N_noise)
            label_train[i] = 1
        for i in tqdm(range(N_sets_train // 3, 2 * N_sets_train // 3)):
            data_train[i] = create_2_circle_noisy(N_points, N_noise)
            label_train[i] = 2
        for i in tqdm(range(2 * N_sets_train // 3, N_sets_train)):
            data_train[i] = create_3_circle_noisy(N_points, N_noise)
            label_train[i] = 3

    shuffler = np.random.permutation(len(data_train))
    label_train = label_train[shuffler]
    data_train = [data_train[p] for p in shuffler]
    if (n_augment_per_sample > 0):
        data_train,label_train = data_augmentation_by_isometries(data_train, label_train, n_augment_per_sample, seed=42)
    return data_train, label_train


############################################
### Computation of persistence diagrams  ###
############################################


def compute_PD(dataset, i):
    u = np.array(dataset[i])
    rcX = gd.AlphaComplex(points=u).create_simplex_tree()
    rcX.persistence()
    dgm = rcX.persistence_intervals_in_dimension(1)
    return dgm

### Definition of our Train/Test sets
Next we use helper functions to compute these point clouds.

In [ ]:
data_train,      label_train       = create_multiple_circles(N_sets_train, N_points, noisy=0, N_noise=N_noise, n_augment_per_sample= 4)
clean_data_test, clean_label_test  = create_multiple_circles(N_sets_test,  N_points, noisy=0, N_noise=N_noise,)
noisy_data_test, noisy_label_test  = create_multiple_circles(N_sets_test,  N_points, noisy=1, N_noise=N_noise)

Let's check some of these point clouds!

#### Plotting Point Clouds

In [ ]:
fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, width_ratios=[1,1,1], wspace=0.0, hspace=0.0)
for i in range(3):
    for j in range(3):
        ax = plt.subplot(gs[i,j])
        ax.scatter(clean_data_test[3*i+j][:,0], clean_data_test[3*i+j][:,1], s=3)
        plt.xticks([])
        plt.yticks([])
plt.savefig('clean_gudhi_pc.png', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, width_ratios=[1,1,1], wspace=0.0, hspace=0.0)
for i in range(3):
    for j in range(3):
        ax = plt.subplot(gs[i,j])
        ax.scatter(noisy_data_test[3*i+j][:,0], noisy_data_test[3*i+j][:,1], s=3)
        plt.xticks([])
        plt.yticks([])
plt.savefig('noisy_gudhi_pc.png', bbox_inches='tight')

#### Cleaning Labels
As one can see, they contain 1 to 3 circular shapes. We will use these circle numbers later for checking the accuracy of the topology computed by RipsNet. For now, let's save them as proper labels.

In [ ]:
le = LabelEncoder().fit(label_train)
label_classif_train = le.transform(label_train)
clean_label_classif_test  = le.transform(clean_label_test)
noisy_label_classif_test  = le.transform(noisy_label_test)

#### Distance Matrix


In [ ]:
def distance_matrix(point_cloud):
    """
    Compute pairwise Euclidean distance matrix for a point cloud.
    Accepts numpy array, list-of-lists, or torch.Tensor of shape (N, d).
    Returns a numpy array of shape (N, N).
    """
    # rely on existing imports: np, torch
    if isinstance(point_cloud, __import__("torch").Tensor):
        point_cloud = point_cloud.cpu().numpy()
    pc = np.asarray(point_cloud, dtype=float)
    if pc.ndim == 1:
        pc = pc.reshape(-1, 1)
    diff = pc[:, None, :] - pc[None, :, :]
    return np.sqrt((diff ** 2).sum(axis=-1))

### Topology of point clouds

In order to train RipsNet, we need the correct topology on the train data. So let's compute it with [Gudhi](https://gudhi.inria.fr/python/latest/). The topology of point clouds can be inferred from several filtrations, so we will use a fast and sparse one (in 2D), namely the [Alpha filtration](https://gudhi.inria.fr/python/latest/alpha_complex_user.html). First, we need to get a sense of the maximal filtration value to use, which we will approximate with the largest pairwise distance of the first 30 point clouds.

In [ ]:
ds = [pairwise_distances(X).flatten() for X in data_train[:30]]
maxd = np.max(np.concatenate(ds))

#### Persistence diagrams
Now, we can compute the persistence diagrams (in homology dimension 1) of the train, clean test and noisy test point clouds!

In [ ]:
PD_train = []
for X in tqdm(data_train):
    st = gd.AlphaComplex(points=X).create_simplex_tree(max_alpha_square=maxd)
    st.persistence()
    dg = st.persistence_intervals_in_dimension(1)
    if len(dg) == 0:
        dg = np.empty([0,2])
    PD_train.append(dg)

In [ ]:
clean_PD_test = []
for X in tqdm(clean_data_test):
    st = gd.AlphaComplex(points=X).create_simplex_tree(max_alpha_square=maxd)
    st.persistence()
    dg = st.persistence_intervals_in_dimension(1)
    if len(dg) == 0:
        dg = np.empty([0,2])
    clean_PD_test.append(dg)

In [ ]:
noisy_PD_test = []
for X in tqdm(noisy_data_test):
    st = gd.AlphaComplex(points=X).create_simplex_tree(max_alpha_square=maxd)
    st.persistence()
    dg = st.persistence_intervals_in_dimension(1)
    if len(dg) == 0:
        dg = np.empty([0,2])
    noisy_PD_test.append(dg)

#### Persistence Image & Persistence Landscape
RipsNet actually works on vectorizations of persistence diagrams (PDs), so we will try two: the [persistence image](https://jmlr.org/papers/volume18/16-337/16-337.pdf), and the [persistence landscape](https://jmlr.csail.mit.edu/papers/volume16/bubenik15a/bubenik15a.pdf). First, let's filter out any PD point with infinite coordinates.

In [ ]:
pds_train      = DiagramSelector(use=True).fit_transform(PD_train)
clean_pds_test = DiagramSelector(use=True).fit_transform(clean_PD_test)
noisy_pds_test = DiagramSelector(use=True).fit_transform(noisy_PD_test)

Fortunately, Gudhi also has code for computing vectorizations!! However, before launching the computations, we have to estimate a few parameters: the image bandwidth (taken as the 20% quantile of the positive birth-persistence-transformed-PD pairwise distances),
and the image and landscape limits (taken as the min and max of birth-persistence-transformed-PD point coordinates).

In [ ]:
vpdtr = np.vstack(pds_train)
pers = vpdtr[:,1]-vpdtr[:,0]
bps_pairs = pairwise_distances(np.hstack([vpdtr[:,0:1],vpdtr[:,1:2]-vpdtr[:,0:1]])[:200]).flatten()
ppers = bps_pairs[np.argwhere(bps_pairs > 1e-5).ravel()]
sigma = np.quantile(ppers, .2)
im_bnds = [np.quantile(vpdtr[:,0],0.), np.quantile(vpdtr[:,0],1.), np.quantile(pers,0.), np.quantile(pers,1.)]
sp_bnds = [np.quantile(vpdtr[:,0],0.), np.quantile(vpdtr[:,1],1.)]

We can now compute and normalize the vectorizations (this can help RipsNet training)!

In [ ]:
if 'im_bnds' not in locals() or im_bnds is None:
    all_points = np.concatenate(pds_train, axis=0)
    x_min, x_max = np.min(all_points[:, 0]), np.max(all_points[:, 0])
    y_min, y_max = np.min(all_points[:, 1]), np.max(all_points[:, 1])
    im_bnds = [x_min, x_max, y_min, y_max]

PI_params = {
    'bandwidth': sigma,
    'weight': lambda x: 10 * np.tanh(x[1]),
    'resolution': [50, 50],
    'im_range': im_bnds
}

PI = PersistenceImage(**PI_params)
PI.fit(pds_train)

PI_train = PI.transform(pds_train)
clean_PI_test = PI.transform(clean_pds_test)
noisy_PI_test = PI.transform(noisy_pds_test)

MPI = np.max(PI_train)
if MPI == 0 or np.isnan(MPI):
    raise ValueError("MPI (max value) is zero or NaN; check persistence diagrams.")
else:
    PI_train /= MPI
    clean_PI_test /= MPI
    noisy_PI_test /= MPI

PI_train = torch.tensor(PI_train, dtype=torch.float32)
clean_PI_test = torch.tensor(clean_PI_test, dtype=torch.float32)
noisy_PI_test = torch.tensor(noisy_PI_test, dtype=torch.float32)

print(f"PI_train shape: {PI_train.shape}")
print(f"clean_PI_test shape: {clean_PI_test.shape}")
print(f"noisy_PI_test shape: {noisy_PI_test.shape}")
print(f"Max pixel intensity (MPI): {MPI:.5f}")


In [ ]:

if 'sp_bnds' not in locals() or sp_bnds is None:
    all_points = np.concatenate(pds_train, axis=0)
    x_min, x_max = np.min(all_points[:, 0]), np.max(all_points[:, 1])
    sp_bnds = [x_min, x_max]


PL_params = {
    'num_landscapes': 5,
    'resolution': 300,
    'sample_range': sp_bnds
}

PL = Landscape(**PL_params)
PL.fit(pds_train)

PL_train = PL.transform(pds_train)
clean_PL_test = PL.transform(clean_pds_test)
noisy_PL_test = PL.transform(noisy_pds_test)

MPL = np.max(PL_train)
if MPL == 0 or np.isnan(MPL):
    raise ValueError("MPL (max landscape value) is zero or NaN; check your persistence diagrams.")
else:
    PL_train /= MPL
    clean_PL_test /= MPL
    noisy_PL_test /= MPL

PL_train = torch.tensor(PL_train, dtype=torch.float32)
clean_PL_test = torch.tensor(clean_PL_test, dtype=torch.float32)
noisy_PL_test = torch.tensor(noisy_PL_test, dtype=torch.float32)

print(f"PL_train shape: {PL_train.shape}")
print(f"clean_PL_test shape: {clean_PL_test.shape}")
print(f"noisy_PL_test shape: {noisy_PL_test.shape}")
print(f"Max landscape value (MPL): {MPL:.5f}")


### Time to Train !
It is finally time to train RipsNet! We will do it with Pytorch, and custom DeepSet operations.

First, let's convert the point clouds into ragged tensors.

In [ ]:
tf_data_train = [torch.tensor(pc, dtype=torch.float32) for pc in data_train]
tf_clean_data_test = [torch.tensor(pc, dtype=torch.float32) for pc in clean_data_test]
tf_noisy_data_test = [torch.tensor(pc, dtype=torch.float32) for pc in noisy_data_test]

Next, we define our RipsNet architecture for learning persistence images, a sequence of fully-connected DeepSet layers, with ReLu activations, Adamax optimizer, and early stopping.

In [ ]:
output_dim = PI_train.shape[1]
model_PI = RaggedPersistenceModel(output_dim=output_dim)

optimizer = Adamax(model_PI.parameters(), lr=5e-4)
criterion = nn.MSELoss()

Time to learn!

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model, optimizer, criterion, train_inputs, train_targets, val_inputs, val_targets, epochs=20, batch_size=32):
    """
    Train a ragged-input model. Returns (best_model, history, best_model_state).
    """
    model = model.to(device)
    patience = 200
    best_val_loss = float('inf')
    patience_counter = 0
    num_epochs = epochs if epochs is not None else 10000
    history = {'train_loss': [], 'val_loss': []}
    best_model_state = None

    # helper to move inputs to device
    def to_device_list(lst):
        out = []
        for x in lst:
            if isinstance(x, torch.Tensor):
                out.append(x.to(device).float())
            else:
                out.append(torch.tensor(x, dtype=torch.float32, device=device))
        return out

    train_inputs = to_device_list(train_inputs)
    val_inputs = to_device_list(val_inputs)

    if isinstance(train_targets, torch.Tensor):
        train_targets = train_targets.to(device).float()
    else:
        train_targets = torch.tensor(train_targets, dtype=torch.float32, device=device)

    if isinstance(val_targets, torch.Tensor):
        val_targets = val_targets.to(device).float()
    else:
        val_targets = torch.tensor(val_targets, dtype=torch.float32, device=device)

    for epoch in range(num_epochs):
        model.train()
        permutation = torch.randperm(len(train_inputs), device=device)
        epoch_loss = 0.0
        for i in range(0, len(train_inputs), batch_size):
            optimizer.zero_grad()
            indices = permutation[i:i + batch_size]
            batch_inputs = [train_inputs[int(idx)].to(device) for idx in indices]
            batch_targets = train_targets[indices]

            outputs = model(batch_inputs)
            if isinstance(outputs, (list, tuple)):
                outputs = outputs[0]
            if not isinstance(outputs, torch.Tensor):
                outputs = torch.tensor(outputs, dtype=batch_targets.dtype, device=device)
            else:
                outputs = outputs.to(device).type(batch_targets.dtype)

            loss = criterion(outputs, batch_targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * len(batch_inputs)

        epoch_loss /= len(train_inputs)
        history['train_loss'].append(epoch_loss)

        model.eval()
        with torch.no_grad():
            val_outputs = model(val_inputs)
            if isinstance(val_outputs, (list, tuple)):
                val_outputs = val_outputs[0]
            val_outputs = val_outputs.to(device).type(val_targets.dtype)
            val_loss = criterion(val_outputs, val_targets).item()
            history['val_loss'].append(val_loss)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.6f}, Val Loss: {val_loss:.6f}')

        if val_loss < best_val_loss - 1e-5:
            best_val_loss = val_loss
            patience_counter = 0
            # save best model (on CPU to avoid holding GPU memory)
            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    # if we saved a best state, load it back into the model on the current device
    if best_model_state is not None:
        model.load_state_dict({k: v.to(device) for k, v in best_model_state.items()})

    return model, history, best_model_state

In [ ]:
# Training RipsNet to predict Persistence Images (original training loop)
model = model_PI
criterion = nn.MSELoss()
optimizer = torch.optim.Adamax(model_PI.parameters(), lr=5e-4)
train_inputs = [torch.tensor(x, dtype=torch.float32).to(device) for x in tf_data_train]
train_targets = torch.tensor(PI_train, dtype=torch.float32).to(device)
val_inputs = [torch.tensor(x, dtype=torch.float32).to(device) for x in tf_clean_data_test]
val_targets = torch.tensor(clean_PI_test, dtype=torch.float32).to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Redefining DenseRagged to correctly handle device placement of dynamically created parameters
class DenseRagged(nn.Module):
    def __init__(self, in_features=None, out_features=30, activation='relu', use_bias=True):
        super(DenseRagged, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.use_bias = use_bias
        self.activation = activation

        # Parameters will be initialized on first forward pass.
        self.weight_param = None # Use distinct names for registered parameters
        self.bias_param = None

    def forward(self, inputs):
        outputs = []
        for x in inputs:
            if self.weight_param is None: # Check if parameters have been initialized
                in_features = x.shape[-1]
                # Initialize weight on the same device as input x
                self.weight_param = nn.Parameter(torch.randn(in_features, self.out_features, device=x.device) * 0.01)
                self.register_parameter('weight_param', self.weight_param) # Register the parameter
                if self.use_bias:
                    # Initialize bias on the same device as input x
                    self.bias_param = nn.Parameter(torch.zeros(self.out_features, device=x.device))
                    self.register_parameter('bias_param', self.bias_param) # Register the parameter

            y = torch.matmul(x, self.weight_param) # Use the registered parameter
            if self.use_bias:
                y = y + self.bias_param # Use the registered parameter
            if self.activation == 'relu':
                y = F.relu(y)
            elif self.activation == 'sigmoid':
                y = torch.sigmoid(y)
            elif self.activation == 'tanh':
                y = torch.tanh(y)
            outputs.append(y)
        return outputs

# Redefining PermopRagged (no changes needed, but must be in scope if RaggedPersistenceModel is redefined)
class PermopRagged(nn.Module):
    def forward(self, inputs):
        return torch.stack([torch.sum(x, dim=0) for x in inputs])

# Redefining RaggedPersistenceModel to use the corrected DenseRagged
class RaggedPersistenceModel(nn.Module):
    def __init__(self, output_dim):
        super(RaggedPersistenceModel, self).__init__()
        self.ragged_layers = nn.ModuleList([
            DenseRagged(out_features=30, activation='relu'),
            DenseRagged(out_features=20, activation='relu'),
            DenseRagged(out_features=10, activation='relu')
        ])
        self.perm = PermopRagged()

        self.fc = nn.Sequential(
            nn.Linear(10, 50),
            nn.ReLU(),
            nn.Linear(50, 100),
            nn.ReLU(),
            nn.Linear(100, 200),
            nn.ReLU(),
            nn.Linear(200, output_dim),
            nn.Sigmoid()
        )

    def forward(self, inputs):
        x = inputs
        for layer in self.ragged_layers:
            x = layer(x)
        x = self.perm(x)
        x = self.fc(x)
        return x

# Re-instantiate model_PI with the new class definition
output_dim = PI_train.shape[1]
model_PI = RaggedPersistenceModel(output_dim=output_dim)

# Re-prepare inputs to avoid UserWarnings and ensure they are on the correct device.
# Assuming tf_data_train, PI_train, tf_clean_data_test, clean_PI_test are correctly defined
# and `device` is set up.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_inputs = [pc.to(device) for pc in tf_data_train]
train_targets = PI_train.to(device)
val_inputs = [pc.to(device) for pc in tf_clean_data_test]
val_targets = clean_PI_test.to(device)

# Redefine optimizer here to use the new model_PI parameters.
optimizer = Adamax(model_PI.parameters(), lr=5e-4)
criterion = nn.MSELoss() # Criterion remains the same

# Ensure num_epochs is defined
try:
    num_epochs
except NameError:
    num_epochs = 10000

model_PI, history_PI, best_model_state = train_model(
    model_PI, optimizer, criterion,
    train_inputs, train_targets,
    val_inputs, val_targets,
    epochs=num_epochs, batch_size=32
)

best_val_loss = min(history_PI['val_loss']) if history_PI.get('val_loss') else float('inf')
print(f"Training complete. Best val_loss = {best_val_loss:.6f}")

Learning is done! The advantage of RipsNet is that we can now use it to predict persistence images directly from point clouds in the blink of an eye!Let's see how accurate the predictions of RipsNet really are, at least on the train set.

In [ ]:
model_PI.eval()

with torch.no_grad():
    # ensure inputs have the same dtype as the model parameters (float32)
    train_inputs = [torch.tensor(x, dtype=torch.float32).to(device) for x in tf_data_train]
    clean_inputs = [torch.tensor(x, dtype=torch.float32).to(device) for x in tf_clean_data_test]
    noisy_inputs = [torch.tensor(x, dtype=torch.float32).to(device) for x in tf_noisy_data_test]

    train_PI_prediction = model_PI(train_inputs).cpu().numpy()
    clean_PI_prediction = model_PI(clean_inputs).cpu().numpy()
    noisy_PI_prediction = model_PI(noisy_inputs).cpu().numpy()

print("Predictions computed:")
print(f"train_PI_prediction shape: {train_PI_prediction.shape}")
print(f"clean_PI_prediction shape: {clean_PI_prediction.shape}")
print(f"noisy_PI_prediction shape: {noisy_PI_prediction.shape}")


Pretty good actually! Let's look at the test set now.

In [ ]:
def to_numpy(x):
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy()
    return np.array(x)

fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, wspace=0.0, hspace=0.0)

for i in range(3):
    for j in range(3):
        idx = 3 * i + j
        ax = plt.subplot(gs[i, j])

        try:
            data = to_numpy(tf_noisy_data_test[idx])
            if data.shape[1] != 2:
                print(f"Skipping idx={idx}, unexpected shape: {data.shape}")
                continue
            ax.scatter(data[:, 0], data[:, 1], s=3)
        except Exception as e:
            print(f"Error at idx={idx} in noisy point cloud: {e}")
            continue

        ax.set_xticks([])
        ax.set_yticks([])

plt.savefig("noisy_gudhi_pc.png", bbox_inches="tight")
print("Saved noisy_gudhi_pc.png")

fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, wspace=0.0, hspace=0.0)

for i in range(3):
    for j in range(3):
        idx = 3 * i + j
        ax = plt.subplot(gs[i, j])

        arr = np.array(noisy_PI_test[idx])
        if arr.size != 50*50:
            print(f"Skipping idx={idx}, unexpected size: {arr.size}")
            continue

        im = np.flip(arr.reshape(50, 50), 0)  # vertical flip
        ax.imshow(im, cmap="jet")
        ax.set_xticks([])
        ax.set_yticks([])

plt.savefig("noisy_gudhi_pi.png", bbox_inches="tight")
print("Saved noisy_gudhi_pi.png")

fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, wspace=0.0, hspace=0.0)

for i in range(3):
    for j in range(3):
        idx = 3 * i + j
        ax = plt.subplot(gs[i, j])

        arr = np.array(noisy_PI_prediction[idx])
        if arr.size != 50*50:
            print(f"Skipping idx={idx}, unexpected size: {arr.size}")
            continue

        pred = np.flip(arr.reshape(50, 50), 0)
        ax.imshow(pred, cmap="jet")
        ax.set_xticks([])
        ax.set_yticks([])

plt.savefig("noisy_ripsnet_pi.png", bbox_inches="tight")
print("Saved noisy_ripsnet_pi.png")


Yep, even though Gudhi now produces bad images because of the ambiant noise, RipsNet predictions still look rather good. Another way to quantify this is to see the difference between classifiers predicting the number of circles that are trained either from Gudhi or from RipsNet predictions. Let's first see how an XGBoost model performs when using Gudhi descriptors.

In [ ]:
model_classif_gudhi = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_gudhi.fit(PI_train, label_classif_train)
clean_test_acc_gudhi = model_classif_gudhi.score(clean_PI_test, clean_label_classif_test)
noisy_test_acc_gudhi = model_classif_gudhi.score(noisy_PI_test, noisy_label_classif_test)

In [ ]:
print("Accuracy on clean test set = " + str(clean_test_acc_gudhi))
print("Accuracy on noisy test set = " + str(noisy_test_acc_gudhi))

Well, the classifier definitely does well on clean data but struggles on noisy data. Let's see how another classifier trained on RipsNet descriptors now performs.

In [ ]:
# ensure labels match the augmented training set size
# `le` was fitted earlier; label_train was augmented to match train inputs
label_classif_train_aug = le.transform(label_train)

model_classif_RN = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_RN.fit(train_PI_prediction, label_classif_train_aug)
clean_test_acc_RN = model_classif_RN.score(clean_PI_prediction, clean_label_classif_test)
noisy_test_acc_RN = model_classif_RN.score(noisy_PI_prediction, noisy_label_classif_test)

In [ ]:
print("Accuracy on clean test set = " + str(clean_test_acc_RN))
print("Accuracy on noisy test set = " + str(noisy_test_acc_RN))

Well, even though the accuracy is slightly worse on clean data, it does remarkably well when noise is added! This illustrates the robustness of RipsNet-based topological descriptors.

Now, let's play the same game with persistence landscapes, using the same architecture (except for the dimension of the final layer of course). In the three following cells, we define RipsNet, train it on landscapes, and use it to predict persistence landscapes from the data.

In [ ]:
output_dim = PL_train.shape[1]
model_PL = RaggedPersistenceModel(output_dim=output_dim)

optimizer = Adamax(model_PL.parameters(), lr=5e-4)
criterion = nn.MSELoss()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_PL = model_PL.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adamax(model_PL.parameters(), lr=5e-4)

patience = 200
best_val_loss = float('inf')
patience_counter = 0
num_epochs = 10000

# prepare data once (move to device)
train_inputs = [x.to(device).float() if isinstance(x, torch.Tensor) else torch.tensor(x, dtype=torch.float32, device=device) for x in tf_data_train]
train_targets = torch.tensor(PL_train, dtype=torch.float32, device=device)

val_inputs = [x.to(device).float() if isinstance(x, torch.Tensor) else torch.tensor(x, dtype=torch.float32, device=device) for x in tf_clean_data_test]
val_targets = torch.tensor(clean_PL_test, dtype=torch.float32, device=device)

history = {'train_loss': [], 'val_loss': []}
best_model_state = {k: v.cpu() for k, v in model_PL.state_dict().items()}

for epoch in range(num_epochs):
    model_PL.train()
    optimizer.zero_grad()

    outputs = model_PL(train_inputs)
    if isinstance(outputs, (list, tuple)):
        outputs = outputs[0]
    if outputs is None:
        raise RuntimeError("model_PL returned None")
    if not isinstance(outputs, torch.Tensor):
        outputs = torch.tensor(outputs, dtype=train_targets.dtype, device=train_targets.device)
    else:
        outputs = outputs.to(train_targets.device).type(train_targets.dtype)

    loss = criterion(outputs, train_targets)
    loss.backward()
    optimizer.step()

    model_PL.eval()
    with torch.no_grad():
        val_outputs = model_PL(val_inputs)
        if isinstance(val_outputs, (list, tuple)):
            val_outputs = val_outputs[0]
        val_outputs = val_outputs.to(val_targets.device).type(val_targets.dtype)
        val_loss = criterion(val_outputs, val_targets)

    history['train_loss'].append(loss.item())
    history['val_loss'].append(val_loss.item())

    if val_loss.item() < best_val_loss - 1e-5:
        best_val_loss = val_loss.item()
        patience_counter = 0
        # save on CPU to avoid GPU memory hold
        best_model_state = {k: v.cpu().clone() for k, v in model_PL.state_dict().items()}
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {loss.item():.6f}, Val loss: {val_loss.item():.6f}")

# load best weights back to device
model_PL.load_state_dict({k: v.to(device) for k, v in best_model_state.items()})
print(f"Training complete. Best val_loss = {best_val_loss:.6f}")


In [ ]:
model_PL.eval()

with torch.no_grad():
    train_inputs = [torch.tensor(x, dtype=torch.float32).to(device) for x in tf_data_train]
    clean_inputs = [torch.tensor(x, dtype=torch.float32).to(device) for x in tf_clean_data_test]
    noisy_inputs = [torch.tensor(x, dtype=torch.float32).to(device) for x in tf_noisy_data_test]

    train_PL_prediction = model_PL(train_inputs).cpu().numpy()
    clean_PL_prediction = model_PL(clean_inputs).cpu().numpy()
    noisy_PL_prediction = model_PL(noisy_inputs).cpu().numpy()

print("Predictions computed:")
print(f"train_PI_prediction shape: {train_PL_prediction.shape}")
print(f"clean_PI_prediction shape: {clean_PL_prediction.shape}")
print(f"noisy_PI_prediction shape: {noisy_PL_prediction.shape}")


Let's see how good RipsNet is for landscapes now.

In [ ]:
def to_numpy(x):
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy()
    return np.array(x)

fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, wspace=0.0, hspace=0.0)

for i in range(3):
    for j in range(3):
        idx = 3 * i + j
        ax = plt.subplot(gs[i, j])

        try:
            data = to_numpy(tf_clean_data_test[idx])
            if data.shape[1] != 2:
                print(f"Skipping idx={idx}, unexpected shape: {data.shape}")
                continue
            ax.scatter(data[:, 0], data[:, 1], s=3)
        except Exception as e:
            print(f"Error at idx={idx} in point cloud: {e}")
            continue

        ax.set_xticks([])
        ax.set_yticks([])

plt.savefig('clean_gudhi_pc.png', bbox_inches='tight')
print("Saved clean_gudhi_pc.png")

fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, wspace=0.0, hspace=0.0)

for i in range(3):
    for j in range(3):
        idx = 3 * i + j
        ax = plt.subplot(gs[i, j])

        arr = np.array(clean_PL_test[idx])
        if arr.size != 50*50:
            print(f"Skipping idx={idx}, unexpected size: {arr.size}")
            continue

        im = np.flip(arr.reshape(50, 50), 0)  # vertical flip
        ax.imshow(im, cmap='jet')
        ax.set_xticks([])
        ax.set_yticks([])

plt.savefig('clean_gudhi_pl.png', bbox_inches='tight')
print("Saved clean_gudhi_pl.png")

fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, wspace=0.0, hspace=0.0)

for i in range(3):
    for j in range(3):
        idx = 3 * i + j
        ax = plt.subplot(gs[i, j])

        arr = np.array(clean_PL_prediction[idx])
        if arr.size != 50*50:
            print(f"Skipping idx={idx}, unexpected size: {arr.size}")
            continue

        pred = np.flip(arr.reshape(50, 50), 0)
        ax.imshow(pred, cmap='jet')
        ax.set_xticks([])
        ax.set_yticks([])

plt.savefig('clean_ripsnet_pl.png', bbox_inches='tight')
print("Saved clean_ripsnet_pl.png")


In [ ]:
def to_numpy(x):
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy()
    return np.array(x)

fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, wspace=0.0, hspace=0.0)

for i in range(3):
    for j in range(3):
        idx = 3 * i + j
        ax = plt.subplot(gs[i, j])

        try:
            data = to_numpy(tf_noisy_data_test[idx])
            if data.shape[1] != 2:
                print(f"Skipping idx={idx}, unexpected shape: {data.shape}")
                continue
            ax.scatter(data[:, 0], data[:, 1], s=3)
        except Exception as e:
            print(f"Error at idx={idx} in noisy point cloud: {e}")
            continue

        ax.set_xticks([])
        ax.set_yticks([])

plt.savefig("noisy_gudhi_pc.png", bbox_inches="tight")
print("Saved noisy_gudhi_pc.png")

fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, wspace=0.0, hspace=0.0)

for i in range(3):
    for j in range(3):
        idx = 3 * i + j
        ax = plt.subplot(gs[i, j])

        arr = np.array(noisy_PL_test[idx])
        if arr.size != 50*50:
            print(f"Skipping idx={idx}, unexpected size: {arr.size}")
            continue

        im = np.flip(arr.reshape(50, 50), 0)  # vertical flip
        ax.imshow(im, cmap="jet")
        ax.set_xticks([])
        ax.set_yticks([])

plt.savefig("noisy_gudhi_pl.png", bbox_inches="tight")
print("Saved noisy_gudhi_pl.png")

fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 3, wspace=0.0, hspace=0.0)

for i in range(3):
    for j in range(3):
        idx = 3 * i + j
        ax = plt.subplot(gs[i, j])

        arr = np.array(noisy_PL_prediction[idx])
        if arr.size != 50*50:
            print(f"Skipping idx={idx}, unexpected size: {arr.size}")
            continue

        pred = np.flip(arr.reshape(50, 50), 0)
        ax.imshow(pred, cmap="jet")
        ax.set_xticks([])
        ax.set_yticks([])

plt.savefig("noisy_ripsnet_pl.png", bbox_inches="tight")
print("Saved noisy_ripsnet_pl.png")


As before, we end by training XGBoost classifier on both Gudhi and RipsNet generated landscapes.

In [ ]:
model_classif_gudhi = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_gudhi.fit(PL_train, label_classif_train)
clean_test_acc_gudhi = model_classif_gudhi.score(clean_PL_test,  clean_label_classif_test)
noisy_test_acc_gudhi = model_classif_gudhi.score(noisy_PL_test,  noisy_label_classif_test)

In [ ]:
print("Accuracy on clean test set = " + str(clean_test_acc_gudhi))
print("Accuracy on noisy test set = " + str(noisy_test_acc_gudhi))

In [ ]:
model_classif_RN = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_RN.fit(train_PL_prediction, label_classif_train)
clean_test_acc_RN = model_classif_RN.score(clean_PL_prediction, clean_label_classif_test)
noisy_test_acc_RN = model_classif_RN.score(noisy_PL_prediction, noisy_label_classif_test)

In [ ]:
print("Accuracy on clean test set = " + str(clean_test_acc_RN))
print("Accuracy on noisy test set = " + str(noisy_test_acc_RN))

The difference is less clear than for persistence images, but present nonetheless! Again, RipsNet exhibits better  robustness properties than Gudhi on noise data, while being faster and almost as good on clean data.

In [ ]:
import torch
import torch.nn as nn
from typing import List

class DistanceMatrixRaggedModel(nn.Module):
    def __init__(self, output_dim, num_points=None, phi_dim=128, rho_hidden=(256,128)):
        """
        output_dim: final descriptor size (e.g., PI_train.shape[1])
        num_points: expected number of points (row length). If None, model infers size at first forward.
        phi_dim: per-row embedding size
        rho_hidden: sizes of hidden layers for global map
        """
        super().__init__()
        self.num_points = num_points
        inp = num_points if num_points is not None else 0
        self._phi_layers = None
        self.phi_dim = phi_dim
        self._build_phi(inp)
        layers = []
        prev = phi_dim
        for h in rho_hidden:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.ReLU())
            prev = h
        layers.append(nn.Linear(prev, output_dim))
        self.rho = nn.Sequential(*layers)

    def _build_phi(self, inp):
        if inp <= 0:
            self._phi_layers = None
            return
        hidden = max(64, self.phi_dim)
        self._phi_layers = nn.Sequential(
            nn.Linear(inp, hidden),
            nn.ReLU(),
            nn.Linear(hidden, self.phi_dim),
            nn.ReLU()
        )

    def forward(self, batch: List[torch.Tensor]):
        """
        batch: list of (Ni x Ni) distance matrices (torch.Tensor)
        returns: (B, output_dim)
        """
        if len(batch) == 0:
            return torch.empty(0, self.rho[-1].out_features, device=next(self.parameters()).device)

        sizes = [m.shape[0] for m in batch]
        max_n = max(sizes)
        device = next(self.parameters()).device

        if self._phi_layers is None or (self.num_points and self.num_points != max_n):
            self._build_phi(max_n)
            self.num_points = max_n
            self._phi_layers = self._phi_layers.to(device)

        B = len(batch)
        mats = torch.zeros((B, max_n, max_n), dtype=torch.float32, device=device)
        row_mask = torch.zeros((B, max_n), dtype=torch.bool, device=device)
        for i, m in enumerate(batch):
            n = m.shape[0]
            mats[i, :n, :n] = m.to(device).float()
            row_mask[i, :n] = 1

        rows = mats.reshape(B * max_n, max_n)
        phi_out = self._phi_layers(rows)
        phi_out = phi_out.reshape(B, max_n, -1)

        lengths = row_mask.sum(dim=1).clamp(min=1).unsqueeze(-1).float()
        summed = (phi_out * row_mask.unsqueeze(-1).float()).sum(dim=1)
        aggregated = summed / lengths

        out = self.rho(aggregated)
        return out


# Task
Compute pairwise Euclidean distance matrices for the training and test point clouds, then initialize and train a DistanceMatrixRaggedModel to predict Persistence Images. Afterwards, evaluate its classification accuracy using an XGBoost classifier on clean and noisy test sets. Repeat this process for Persistence Landscapes.

## Compute distance matrices

### Subtask:
Compute pairwise Euclidean distance matrices for the training and test point clouds.


**Reasoning**:
I will apply the `distance_matrix` function to each point cloud in `data_train`, `clean_data_test`, and `noisy_data_test` to compute the pairwise Euclidean distance matrices for the respective datasets. I will use `tqdm` to display progress during the computation.



In [ ]:
dm_train = []
for X in tqdm(data_train, desc='Computing DM for training data'):
    dm_train.append(distance_matrix(X))

dm_clean_test = []
for X in tqdm(clean_data_test, desc='Computing DM for clean test data'):
    dm_clean_test.append(distance_matrix(X))

dm_noisy_test = []
for X in tqdm(noisy_data_test, desc='Computing DM for noisy test data'):
    dm_noisy_test.append(distance_matrix(X))

print(f"Shape of first training distance matrix: {dm_train[0].shape}")
print(f"Shape of first clean test distance matrix: {dm_clean_test[0].shape}")
print(f"Shape of first noisy test distance matrix: {dm_noisy_test[0].shape}")

**Reasoning**:
I will apply the `distance_matrix` function to each point cloud in `data_train`, `clean_data_test`, and `noisy_data_test` to compute the pairwise Euclidean distance matrices for the respective datasets. I will use `tqdm` to display progress during the computation.



In [ ]:
dm_train = []
for X in tqdm(data_train, desc='Computing DM for training data'):
    dm_train.append(distance_matrix(X))

dm_clean_test = []
for X in tqdm(clean_data_test, desc='Computing DM for clean test data'):
    dm_clean_test.append(distance_matrix(X))

dm_noisy_test = []
for X in tqdm(noisy_data_test, desc='Computing DM for noisy test data'):
    dm_noisy_test.append(distance_matrix(X))

print(f"Shape of first training distance matrix: {dm_train[0].shape}")
print(f"Shape of first clean test distance matrix: {dm_clean_test[0].shape}")
print(f"Shape of first noisy test distance matrix: {dm_noisy_test[0].shape}")

**Reasoning**:
The previous code block was already executed and completed the task of computing distance matrices. The next step is to initialize the `DistanceMatrixRaggedModel` to predict Persistence Images.



In [ ]:
output_dim_pi = PI_train.shape[1]
model_dm_pi = DistanceMatrixRaggedModel(output_dim=output_dim_pi)

print(f"Initialized DistanceMatrixRaggedModel for PI with output_dim: {output_dim_pi}")

**Reasoning**:
The next step is to train the `DistanceMatrixRaggedModel` for Persistence Images using the pre-computed distance matrices and Persistence Images. I will reuse the `train_model` function defined previously for training with early stopping.



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_dm_pi.to(device)
criterion = nn.MSELoss()
optimizer = Adamax(model_dm_pi.parameters(), lr=5e-4)

# Ensure num_epochs is defined
try:
    num_epochs
except NameError:
    num_epochs = 10000

model_dm_pi, history_dm_pi, best_model_state_dm_pi = train_model(
    model_dm_pi, optimizer, criterion,
    dm_train, PI_train,
    dm_clean_test, clean_PI_test,
    epochs=num_epochs, batch_size=32
)

best_val_loss_dm_pi = min(history_dm_pi['val_loss']) if history_dm_pi.get('val_loss') else float('inf')
print(f"Training DistanceMatrixRaggedModel for PI complete. Best val_loss = {best_val_loss_dm_pi:.6f}")

## Evaluate classification accuracy (PI)

### Subtask:
Make predictions with the trained `DistanceMatrixRaggedModel` for Persistence Images on clean and noisy test sets. Then, train an XGBoost classifier on these predictions and report the classification accuracy.


**Reasoning**:
I will make predictions using the trained `DistanceMatrixRaggedModel` on the distance matrices for training, clean test, and noisy test data. Then, I will train an XGBoost classifier on these predictions and report its accuracy on both clean and noisy test sets.



In [ ]:
model_dm_pi.eval()

with torch.no_grad():
    # Ensure inputs are on the correct device and dtype
    # The train_model function already converted dm_train and dm_clean_test to lists of tensors on the device.
    # So we only need to handle dm_noisy_test explicitly here if it wasn't already handled for validation.
    # However, to be safe and consistent with the train_model's input preparation for evaluation,
    # let's re-convert them.
    dm_train_tensor = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_train]
    dm_clean_test_tensor = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_clean_test]
    dm_noisy_test_tensor = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_noisy_test]

    dm_train_PI_prediction = model_dm_pi(dm_train_tensor).cpu().numpy()
    dm_clean_PI_prediction = model_dm_pi(dm_clean_test_tensor).cpu().numpy()
    dm_noisy_PI_prediction = model_dm_pi(dm_noisy_test_tensor).cpu().numpy()

print("Predictions computed for DistanceMatrixRaggedModel for PI:")
print(f"dm_train_PI_prediction shape: {dm_train_PI_prediction.shape}")
print(f"dm_clean_PI_prediction shape: {dm_clean_PI_prediction.shape}")
print(f"dm_noisy_PI_prediction shape: {dm_noisy_PI_prediction.shape}")

# Train an XGBoost classifier on the predictions
model_classif_dm_pi = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_dm_pi.fit(dm_train_PI_prediction, label_classif_train)

# Evaluate accuracy on clean test set
clean_test_acc_dm_pi = model_classif_dm_pi.score(dm_clean_PI_prediction, clean_label_classif_test)
print(f"Accuracy on clean test set : {clean_test_acc_dm_pi}")

# Evaluate accuracy on noisy test set
noisy_test_acc_dm_pi = model_classif_dm_pi.score(dm_noisy_PI_prediction, noisy_label_classif_test)
print(f"Accuracy on noisy test set : {noisy_test_acc_dm_pi}")

**Reasoning**:
The previous code block successfully executed. The next step is to make predictions using the trained `DistanceMatrixRaggedModel` for Persistence Images on the distance matrices for training, clean test, and noisy test data. Then, an XGBoost classifier will be trained on these predictions, and its accuracy will be reported on both clean and noisy test sets, as requested by the subtask.



In [ ]:
model_dm_pi.eval()

with torch.no_grad():
    # Ensure inputs are on the correct device and dtype
    # The train_model function already converted dm_train and dm_clean_test to lists of tensors on the device.
    # So we only need to handle dm_noisy_test explicitly here if it wasn't already handled for validation.
    # However, to be safe and consistent with the train_model's input preparation for evaluation,
    # let's re-convert them.
    dm_train_tensor = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_train]
    dm_clean_test_tensor = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_clean_test]
    dm_noisy_test_tensor = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_noisy_test]

    dm_train_PI_prediction = model_dm_pi(dm_train_tensor).cpu().numpy()
    dm_clean_PI_prediction = model_dm_pi(dm_clean_test_tensor).cpu().numpy()
    dm_noisy_PI_prediction = model_dm_pi(dm_noisy_test_tensor).cpu().numpy()

print("Predictions computed for DistanceMatrixRaggedModel for PI:")
print(f"dm_train_PI_prediction shape: {dm_train_PI_prediction.shape}")
print(f"dm_clean_PI_prediction shape: {dm_clean_PI_prediction.shape}")
print(f"dm_noisy_PI_prediction shape: {dm_noisy_PI_prediction.shape}")

# Train an XGBoost classifier on the predictions
model_classif_dm_pi = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_dm_pi.fit(dm_train_PI_prediction, label_classif_train)

# Evaluate accuracy on clean test set
clean_test_acc_dm_pi = model_classif_dm_pi.score(dm_clean_PI_prediction, clean_label_classif_test)
print(f"Accuracy on clean test set (DistanceMatrixRaggedModel for PI): {clean_test_acc_dm_pi}")

# Evaluate accuracy on noisy test set
noisy_test_acc_dm_pi = model_classif_dm_pi.score(dm_noisy_PI_prediction, noisy_label_classif_test)
print(f"Accuracy on noisy test set (DistanceMatrixRaggedModel for PI): {noisy_test_acc_dm_pi}")

## Train DistanceMatrixRaggedModel for Persistence Landscapes

### Subtask:
Initialize, train, and make predictions with a new `DistanceMatrixRaggedModel` using the computed distance matrices and Persistence Landscapes as targets.


**Reasoning**:
I will initialize a `DistanceMatrixRaggedModel` for Persistence Landscapes with the correct output dimension, then set up the optimizer and loss function. Afterwards, I will use the `train_model` function to train `model_dm_pl` using the distance matrices and Persistence Landscapes as targets. Finally, I will put the model in evaluation mode and make predictions.



In [ ]:
output_dim_pl = PL_train.shape[1]
model_dm_pl = DistanceMatrixRaggedModel(output_dim=output_dim_pl)

print(f"Initialized DistanceMatrixRaggedModel for PL with output_dim: {output_dim_pl}")

model_dm_pl.to(device)
criterion_pl = nn.MSELoss()
optimizer_pl = Adamax(model_dm_pl.parameters(), lr=5e-4)

# Ensure num_epochs is defined
try:
    num_epochs
except NameError:
    num_epochs = 10000

model_dm_pl, history_dm_pl, best_model_state_dm_pl = train_model(
    model_dm_pl, optimizer_pl, criterion_pl,
    dm_train, PL_train,
    dm_clean_test, clean_PL_test,
    epochs=num_epochs, batch_size=32
)

best_val_loss_dm_pl = min(history_dm_pl['val_loss']) if history_dm_pl.get('val_loss') else float('inf')
print(f"Training DistanceMatrixRaggedModel for PL complete. Best val_loss = {best_val_loss_dm_pl:.6f}")

model_dm_pl.eval()

with torch.no_grad():
    # Ensure inputs are on the correct device and dtype for prediction
    dm_train_tensor_pl = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_train]
    dm_clean_test_tensor_pl = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_clean_test]
    dm_noisy_test_tensor_pl = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_noisy_test]

    dm_train_PL_prediction = model_dm_pl(dm_train_tensor_pl).cpu().numpy()
    dm_clean_PL_prediction = model_dm_pl(dm_clean_test_tensor_pl).cpu().numpy()
    dm_noisy_PL_prediction = model_dm_pl(dm_noisy_test_tensor_pl).cpu().numpy()

print("Predictions computed for DistanceMatrixRaggedModel for PL:")
print(f"dm_train_PL_prediction shape: {dm_train_PL_prediction.shape}")
print(f"dm_clean_PL_prediction shape: {dm_clean_PL_prediction.shape}")
print(f"dm_noisy_PL_prediction shape: {dm_noisy_PL_prediction.shape}")

In [ ]:
model_dm_pl.eval()

with torch.no_grad():
    # Ensure inputs are on the correct device and dtype
    # The train_model function already converted dm_train and dm_clean_test to lists of tensors on the device.
    # So we only need to handle dm_noisy_test explicitly here if it wasn't already handled for validation.
    # However, to be safe and consistent with the train_model's input preparation for evaluation,
    # let's re-convert them.
    dm_train_tensor = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_train]
    dm_clean_test_tensor = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_clean_test]
    dm_noisy_test_tensor = [torch.tensor(dm, dtype=torch.float32).to(device) for dm in dm_noisy_test]

    dm_train_PL_prediction = model_dm_pl(dm_train_tensor).cpu().numpy()
    dm_clean_PL_prediction = model_dm_pl(dm_clean_test_tensor).cpu().numpy()
    dm_noisy_PL_prediction = model_dm_pl(dm_noisy_test_tensor).cpu().numpy()

print("Predictions computed for DistanceMatrixRaggedModel for PL:")
print(f"dm_train_PL_prediction shape: {dm_train_PL_prediction.shape}")
print(f"dm_clean_PL_prediction shape: {dm_clean_PL_prediction.shape}")
print(f"dm_noisy_PL_prediction shape: {dm_noisy_PL_prediction.shape}")

# Train an XGBoost classifier on the predictions
model_classif_dm_pl = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_dm_pl.fit(dm_train_PL_prediction, label_classif_train)

# Evaluate accuracy on clean test set
clean_test_acc_dm_pl = model_classif_dm_pl.score(dm_clean_PL_prediction, clean_label_classif_test)
print(f"Accuracy on clean test set (DistanceMatrixRaggedModel for PI): {clean_test_acc_dm_pi}")

# Evaluate accuracy on noisy test set
noisy_test_acc_dm_pl = model_classif_dm_pl.score(dm_noisy_PL_prediction, noisy_label_classif_test)
print(f"Accuracy on noisy test set (DistanceMatrixRaggedModel for PI): {noisy_test_acc_dm_pi}")

## Define PointNet Model Class

### Subtask:
Create a new `nn.Module` class named `PointNet` that implements a PointNet-like architecture, accepting a list of 2D point clouds, padding them internally, processing with shared MLPs and max-pooling, and outputting a fixed-size vector.


**Reasoning**:
I need to implement the `PointNet` class as described, including shared MLPs (phi), max-pooling, and global MLPs (rho), with internal padding for variable-sized point clouds.



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from typing import List

class PointNet(nn.Module):
    def __init__(self, output_dim, phi_dims=[64, 128, 256], rho_dims=[256, 128]):
        super().__init__()

        # Phi function (shared MLP) - maps each point to a feature vector
        phi_layers = []
        in_features = 2 # 2D points
        for dim in phi_dims:
            phi_layers.append(nn.Linear(in_features, dim))
            phi_layers.append(nn.ReLU())
            in_features = dim
        self.phi_layers = nn.Sequential(*phi_layers)

        # Rho function (global MLP) - processes aggregated features
        rho_layers = []
        in_features = phi_dims[-1] # Input to rho is the max-pooled feature from phi
        for dim in rho_dims:
            rho_layers.append(nn.Linear(in_features, dim))
            rho_layers.append(nn.ReLU())
            in_features = dim
        rho_layers.append(nn.Linear(in_features, output_dim))
        # No activation on the final layer for regression tasks
        self.rho_layers = nn.Sequential(*rho_layers)

    def forward(self, batch: List[torch.Tensor]):
        if len(batch) == 0:
            # Return an empty tensor with the correct output dimension if batch is empty
            return torch.empty(0, self.rho_layers[-1].out_features, device=next(self.parameters()).device)

        # Pad point clouds to the maximum length in the batch
        # `pad_sequence` expects a list of tensors of shape (L, *) where L is sequence length
        padded_batch = pad_sequence(batch, batch_first=True, padding_value=0.0)
        # original_lengths = [len(pc) for pc in batch]

        # Create a mask for valid points
        # Shape: (batch_size, max_num_points)
        mask = torch.zeros(padded_batch.shape[0], padded_batch.shape[1], dtype=torch.bool, device=padded_batch.device)
        for i, pc in enumerate(batch):
            mask[i, :len(pc)] = True

        # Apply shared MLP (phi function) to all points
        # Reshape to (total_num_points_in_batch, input_dim) for MLP, then reshape back
        # phi_out shape: (batch_size, max_num_points, phi_dims[-1])
        phi_out = self.phi_layers(padded_batch.view(-1, padded_batch.shape[-1]))
        phi_out = phi_out.view(padded_batch.shape[0], padded_batch.shape[1], -1)

        # Apply mask before max-pooling
        # Set features of padded points to a very small negative value
        # so they don't affect the max operation (especially with ReLU later in rho)
        min_val = torch.finfo(phi_out.dtype).min
        masked_phi_out = phi_out.masked_fill(~mask.unsqueeze(-1), min_val)

        # Permutation-invariant aggregation (max-pooling over points)
        # aggregated_features shape: (batch_size, phi_dims[-1])
        aggregated_features, _ = torch.max(masked_phi_out, dim=1)

        # Apply global MLP (rho function)
        # output shape: (batch_size, output_dim)
        output = self.rho_layers(aggregated_features)

        return output

print("PointNet class defined.")

## Initialize and Train PointNet for Persistence Images

### Subtask:
Initialize an instance of the `PointNet` model with `output_dim` set to match the dimension of `PI_train`. Configure `MSELoss` as the criterion and `Adamax` as the optimizer. Then, train the `PointNet` model using `tf_data_train` (the original point clouds) as inputs and `PI_train` as targets. Use `tf_clean_data_test` and `clean_PI_test` for validation, leveraging the `train_model` helper function.


**Reasoning**:
I will initialize the `PointNet` model, define the loss function and optimizer, and then train the model using the `train_model` helper function with `tf_data_train` as inputs and `PI_train` as targets, as specified in the subtask.



In [ ]:
output_dim_pi_pn = PI_train.shape[1]
model_PN_PI = PointNet(output_dim=output_dim_pi_pn)

print(f"Initialized PointNet for PI with output_dim: {output_dim_pi_pn}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_PN_PI.to(device)
criterion_pn = nn.MSELoss()
optimizer_pn = Adamax(model_PN_PI.parameters(), lr=5e-4)

# Ensure num_epochs is defined
try:
    num_epochs
except NameError:
    num_epochs = 10000

model_PN_PI, history_PN_PI, best_model_state_pn_pi = train_model(
    model_PN_PI, optimizer_pn, criterion_pn,
    tf_data_train, PI_train,
    tf_clean_data_test, clean_PI_test,
    epochs=num_epochs, batch_size=32
)

best_val_loss_pn_pi = min(history_PN_PI['val_loss']) if history_PN_PI.get('val_loss') else float('inf')
print(f"Training PointNet for PI complete. Best val_loss = {best_val_loss_pn_pi:.6f}")

**Reasoning**:
Now that the `PointNet` model for Persistence Images has been trained, I need to use it to make predictions on the training, clean test, and noisy test point clouds. These predictions will then be used to train and evaluate an `XGBoost` classifier to assess the model's performance on topology prediction.



In [ ]:
model_PN_PI.eval()

with torch.no_grad():
    # Ensure inputs are on the correct device and dtype
    tf_data_train_tensor = [torch.tensor(pc, dtype=torch.float32).to(device) for pc in tf_data_train]
    tf_clean_data_test_tensor = [torch.tensor(pc, dtype=torch.float32).to(device) for pc in tf_clean_data_test]
    tf_noisy_data_test_tensor = [torch.tensor(pc, dtype=torch.float32).to(device) for pc in tf_noisy_data_test]

    pn_train_PI_prediction = model_PN_PI(tf_data_train_tensor).cpu().numpy()
    pn_clean_PI_prediction = model_PN_PI(tf_clean_data_test_tensor).cpu().numpy()
    pn_noisy_PI_prediction = model_PN_PI(tf_noisy_data_test_tensor).cpu().numpy()

print("Predictions computed for PointNet for PI:")
print(f"pn_train_PI_prediction shape: {pn_train_PI_prediction.shape}")
print(f"pn_clean_PI_prediction shape: {pn_clean_PI_prediction.shape}")
print(f"pn_noisy_PI_prediction shape: {pn_noisy_PI_prediction.shape}")

# Train an XGBoost classifier on the predictions from PointNet (PI)
model_classif_PN_PI = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_PN_PI.fit(pn_train_PI_prediction, label_classif_train)

# Evaluate accuracy on clean test set
clean_test_acc_PN_PI = model_classif_PN_PI.score(pn_clean_PI_prediction, clean_label_classif_test)
print(f"Accuracy on clean test set (PointNet for PI): {clean_test_acc_PN_PI}")

# Evaluate accuracy on noisy test set
noisy_test_acc_PN_PI = model_classif_PN_PI.score(pn_noisy_PI_prediction, noisy_label_classif_test)
print(f"Accuracy on noisy test set (PointNet for PI): {noisy_test_acc_PN_PI}")

**Reasoning**:
The previous code block generated `UserWarning` messages because it attempted to create new tensors from existing tensors. To resolve this, I will modify the input preparation lines to directly move the existing tensors to the specified device, avoiding unnecessary tensor construction and adhering to best practices for tensor handling.



In [ ]:
model_PN_PI.eval()

with torch.no_grad():
    # Ensure inputs are on the correct device and dtype
    # tf_data_train, tf_clean_data_test, and tf_noisy_data_test are already lists of torch.Tensors
    # created with dtype=torch.float32, so we just need to move them to the device.
    tf_data_train_on_device = [pc.to(device) for pc in tf_data_train]
    tf_clean_data_test_on_device = [pc.to(device) for pc in tf_clean_data_test]
    tf_noisy_data_test_on_device = [pc.to(device) for pc in tf_noisy_data_test]

    pn_train_PI_prediction = model_PN_PI(tf_data_train_on_device).cpu().numpy()
    pn_clean_PI_prediction = model_PN_PI(tf_clean_data_test_on_device).cpu().numpy()
    pn_noisy_PI_prediction = model_PN_PI(tf_noisy_data_test_on_device).cpu().numpy()

print("Predictions computed for PointNet for PI:")
print(f"pn_train_PI_prediction shape: {pn_train_PI_prediction.shape}")
print(f"pn_clean_PI_prediction shape: {pn_clean_PI_prediction.shape}")
print(f"pn_noisy_PI_prediction shape: {pn_noisy_PI_prediction.shape}")

# Train an XGBoost classifier on the predictions from PointNet (PI)
model_classif_PN_PI = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_PN_PI.fit(pn_train_PI_prediction, label_classif_train)

# Evaluate accuracy on clean test set
clean_test_acc_PN_PI = model_classif_PN_PI.score(pn_clean_PI_prediction, clean_label_classif_test)
print(f"Accuracy on clean test set (PointNet for PI): {clean_test_acc_PN_PI}")

# Evaluate accuracy on noisy test set
noisy_test_acc_PN_PI = model_classif_PN_PI.score(pn_noisy_PI_prediction, noisy_label_classif_test)
print(f"Accuracy on noisy test set (PointNet for PI): {noisy_test_acc_PN_PI}")

**Reasoning**:
I need to initialize a `PointNet` model for Persistence Landscapes, train it using the `train_model` helper function, and then make predictions on the training and test sets as the initial step for evaluating its classification accuracy.



In [ ]:
output_dim_pl_pn = PL_train.shape[1]
model_PN_PL = PointNet(output_dim=output_dim_pl_pn)

print(f"Initialized PointNet for PL with output_dim: {output_dim_pl_pn}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_PN_PL.to(device)
criterion_pn_pl = nn.MSELoss()
optimizer_pn_pl = Adamax(model_PN_PL.parameters(), lr=5e-4)

# Ensure num_epochs is defined
try:
    num_epochs
except NameError:
    num_epochs = 10000

model_PN_PL, history_PN_PL, best_model_state_pn_pl = train_model(
    model_PN_PL, optimizer_pn_pl, criterion_pn_pl,
    tf_data_train, PL_train,
    tf_clean_data_test, clean_PL_test,
    epochs=num_epochs, batch_size=32
)

best_val_loss_pn_pl = min(history_PN_PL['val_loss']) if history_PN_PL.get('val_loss') else float('inf')
print(f"Training PointNet for PL complete. Best val_loss = {best_val_loss_pn_pl:.6f}")

model_PN_PL.eval()

with torch.no_grad():
    # Ensure inputs are on the correct device and dtype
    tf_data_train_on_device = [pc.to(device) for pc in tf_data_train]
    tf_clean_data_test_on_device = [pc.to(device) for pc in tf_clean_data_test]
    tf_noisy_data_test_on_device = [pc.to(device) for pc in tf_noisy_data_test]

    pn_train_PL_prediction = model_PN_PL(tf_data_train_on_device).cpu().numpy()
    pn_clean_PL_prediction = model_PN_PL(tf_clean_data_test_on_device).cpu().numpy()
    pn_noisy_PL_prediction = model_PN_PL(tf_noisy_data_test_on_device).cpu().numpy()

print("Predictions computed for PointNet for PL:")
print(f"pn_train_PL_prediction shape: {pn_train_PL_prediction.shape}")
print(f"pn_clean_PL_prediction shape: {pn_clean_PL_prediction.shape}")
print(f"pn_noisy_PL_prediction shape: {pn_noisy_PL_prediction.shape}")

**Reasoning**:
Now that the `PointNet` model for Persistence Landscapes has been trained and predictions generated, I need to train an `XGBoost` classifier using these predictions and evaluate its classification accuracy on the clean and noisy test sets to assess the model's performance.



In [ ]:
model_classif_PN_PL = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_PN_PL.fit(pn_train_PL_prediction, label_classif_train)

# Evaluate accuracy on clean test set
clean_test_acc_PN_PL = model_classif_PN_PL.score(pn_clean_PL_prediction, clean_label_classif_test)
print(f"Accuracy on clean test set (PointNet for PL): {clean_test_acc_PN_PL}")

# Evaluate accuracy on noisy test set
noisy_test_acc_PN_PL = model_classif_PN_PL.score(pn_noisy_PL_prediction, noisy_label_classif_test)
print(f"Accuracy on noisy test set (PointNet for PL): {noisy_test_acc_PN_PL}")

**Reasoning**:
Now that the `PointNet` model for Persistence Landscapes has been trained and predictions generated, I need to train an `XGBoost` classifier using these predictions and evaluate its classification accuracy on the clean and noisy test sets to assess the model's performance.



In [ ]:
model_classif_PN_PL = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
model_classif_PN_PL.fit(pn_train_PL_prediction, label_classif_train)

# Evaluate accuracy on clean test set
clean_test_acc_PN_PL = model_classif_PN_PL.score(pn_clean_PL_prediction, clean_label_classif_test)
print(f"Accuracy on clean test set (PointNet for PL): {clean_test_acc_PN_PL}")

# Evaluate accuracy on noisy test set
noisy_test_acc_PN_PL = model_classif_PN_PL.score(pn_noisy_PL_prediction, noisy_label_classif_test)
print(f"Accuracy on noisy test set (PointNet for PL): {noisy_test_acc_PN_PL}")

In [ ]:
!pip install POT

In [ ]:
import gudhi as gd
import numpy as np
from tqdm import tqdm
# Explicitly import wasserstein submodule
import gudhi.wasserstein as gdw

def compute_isometry_robustness_score(pc, model, n_augment_per_score, seed=42):
    """
    Calculates the average Wasserstein distance between the persistence diagram of an original
    point cloud and those of its isometrically augmented versions.
    """
    rng = np.random.default_rng(seed)

    # 1. Compute PD for the original point cloud
    st_original = gd.AlphaComplex(points=pc).create_simplex_tree(max_alpha_square=maxd)
    st_original.persistence()
    pd_original = st_original.persistence_intervals_in_dimension(1)
    if len(pd_original) == 0:
        pd_original = np.empty([0, 2])

    # 2. Generate augmented point clouds
    augmented_pcs = augment_isometries(pc, n_augment_per_score, rng)

    PD_train_prediction = model(pc).cpu()
    PD_clean_prediction = model(pc).cpu()
    PD_noisy_prediction = model(pc).cpu()

    wasserstein_distances = []
    for aug_pc in augmented_pcs:
        # 3. Compute PD for each augmented point cloud
        pd_augmented = model(aug_pc).cpu()
        if len(pd_augmented) == 0:
            pd_augmented = np.empty([0, 2])

        # 4. Calculate 1-Wasserstein distance
        # Ensure diagrams are 2D arrays, even if empty
        dist = gdw.wasserstein_distance(pd_original, pd_augmented, order=1, enable_autodiff=False)
        wasserstein_distances.append(dist)

    # 5. Return the mean of these distances
    if len(wasserstein_distances) > 0:
        return np.mean(wasserstein_distances)
    else:
        return 0.0

print("compute_isometry_robustness_score function defined.")

In [ ]:
clean_isometry_robustness_scores = []
for pc in tqdm(clean_data_test, desc='Computing isometry robustness for clean data'):
    score = compute_isometry_robustness_score(pc,model_PN_PI, n_augment_per_score=5, seed=42)
    clean_isometry_robustness_scores.append(score)

noisy_isometry_robustness_scores = []
for pc in tqdm(noisy_data_test, desc='Computing isometry robustness for noisy data'):
    score = compute_isometry_robustness_score(pc,model_PN_PI, n_augment_per_score=5, seed=42)
    noisy_isometry_robustness_scores.append(score)

print(f"Average isometry robustness score for clean data: {np.mean(clean_isometry_robustness_scores):.4f}")
print(f"Average isometry robustness score for noisy data: {np.mean(noisy_isometry_robustness_scores):.4f}")

## Modify compute_isometry_robustness_score for vector output

### Subtask:
Directly modify the `compute_isometry_robustness_score` function to calculate the average Euclidean (L2) distance between a model's vector predictions for an original point cloud and its isometrically augmented versions. This involves converting point clouds to PyTorch tensors, moving them to the correct device, and using `np.linalg.norm` for distance calculation.


**Reasoning**:
The user wants to modify the `compute_isometry_robustness_score` function to calculate the average Euclidean (L2) distance between a model's vector predictions for an original point cloud and its isometrically augmented versions. This involves converting point clouds to PyTorch tensors and moving them to the correct device. I will replace the existing function with the updated version that takes the point cloud (numpy array), converts it to a tensor before passing it to the model, and then calculates the L2 distance between the model's output vectors.



In [ ]:
import gudhi as gd
import numpy as np
from tqdm import tqdm
import torch

# Ensure DistanceMatrixRaggedModel, RaggedPersistenceModel, and PointNet are defined in the global scope
# (Assuming they are already defined in previous cells or imported)

def compute_isometry_robustness_score(pc, model, n_augment_per_score, device, seed=42):
    """
    Calculates the average Euclidean (L2) distance between the vector prediction of an original
    point cloud and those of its isometrically augmented versions.
    It adapts its input based on the model type (PointNet/RaggedPersistenceModel vs DistanceMatrixRaggedModel).
    """
    rng = np.random.default_rng(seed)
    model.eval()

    # Determine input type based on model class
    # Check if the model is an instance of DistanceMatrixRaggedModel (assuming it's defined)
    is_distance_matrix_model = False
    if 'DistanceMatrixRaggedModel' in globals() and isinstance(model, globals()['DistanceMatrixRaggedModel']):
        is_distance_matrix_model = True

    # Prepare input for the original point cloud
    if is_distance_matrix_model:
        # Convert point cloud to distance matrix
        dm_original = distance_matrix(pc)
        input_original = [torch.tensor(dm_original, dtype=torch.float32).to(device)]
    else:
        # Use point cloud directly
        input_original = [torch.tensor(pc, dtype=torch.float32).to(device)]

    # 1. Compute vector output for the original point cloud
    with torch.no_grad():
        pred_original = model(input_original)
        # Ensure output is flattened if it comes as a list or has a batch dimension of 1
        if isinstance(pred_original, (list, tuple)):
            pred_original = pred_original[0]
        pred_original = pred_original.cpu().numpy().flatten()

    # 2. Generate augmented point clouds
    augmented_pcs = augment_isometries(pc, n_augment_per_score, rng)

    euclidean_distances = []
    for aug_pc in augmented_pcs:
        # Prepare input for augmented point cloud
        if is_distance_matrix_model:
            dm_augmented = distance_matrix(aug_pc)
            input_augmented = [torch.tensor(dm_augmented, dtype=torch.float32).to(device)]
        else:
            input_augmented = [torch.tensor(aug_pc, dtype=torch.float32).to(device)]

        # 3. Compute vector output for each augmented point cloud
        with torch.no_grad():
            pred_augmented = model(input_augmented)
            if isinstance(pred_augmented, (list, tuple)):
                pred_augmented = pred_augmented[0]
            pred_augmented = pred_augmented.cpu().numpy().flatten()

        # 4. Calculate L2 (Euclidean) distance
        dist = np.linalg.norm(pred_original - pred_augmented)
        euclidean_distances.append(dist)

    # 5. Return the mean of these distances
    if len(euclidean_distances) > 0:
        return np.mean(euclidean_distances)
    else:
        return 0.0

print("compute_isometry_robustness_score function redefined to handle different model inputs.")

**Reasoning**:
The `compute_isometry_robustness_score` function has been updated to correctly handle different model input types. Now, I need to re-execute the code block that calculates and prints the isometry robustness scores for all models (RipsNet, DistanceMatrixRaggedModel, and PointNet) for both Persistence Images and Persistence Landscapes on clean and noisy test data. This should now run without errors and provide the desired performance metrics.



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Calculate Isometry Robustness for RipsNet PI Predictions
clean_ripsnet_pi_isometry_robustness_scores = []
for pc in tqdm(clean_data_test, desc='Computing RipsNet PI isometry robustness for clean data'):
    score = compute_isometry_robustness_score(pc, model_PI, n_augment_per_score=5, device=device, seed=42)
    clean_ripsnet_pi_isometry_robustness_scores.append(score)

noisy_ripsnet_pi_isometry_robustness_scores = []
for pc in tqdm(noisy_data_test, desc='Computing RipsNet PI isometry robustness for noisy data'):
    score = compute_isometry_robustness_score(pc, model_PI, n_augment_per_score=5, device=device, seed=42)
    noisy_ripsnet_pi_isometry_robustness_scores.append(score)

print(f"Average RipsNet PI isometry robustness score for clean data: {np.mean(clean_ripsnet_pi_isometry_robustness_scores):.6f}")
print(f"Standard deviation RipsNet PI isometry robustness score for clean data: {np.std(clean_ripsnet_pi_isometry_robustness_scores):.6f}")
print(f"Average RipsNet PI isometry robustness score for noisy data: {np.mean(noisy_ripsnet_pi_isometry_robustness_scores):.6f}")
print(f"Standard deviation RipsNet PI isometry robustness score for noisy data: {np.std(noisy_ripsnet_pi_isometry_robustness_scores):.6f}")

# Calculate Isometry Robustness for RipsNet PL Predictions
clean_ripsnet_pl_isometry_robustness_scores = []
for pc in tqdm(clean_data_test, desc='Computing RipsNet PL isometry robustness for clean data'):
    score = compute_isometry_robustness_score(pc, model_PL, n_augment_per_score=5, device=device, seed=42)
    clean_ripsnet_pl_isometry_robustness_scores.append(score)

noisy_ripsnet_pl_isometry_robustness_scores = []
for pc in tqdm(noisy_data_test, desc='Computing RipsNet PL isometry robustness for noisy data'):
    score = compute_isometry_robustness_score(pc, model_PL, n_augment_per_score=5, device=device, seed=42)
    noisy_ripsnet_pl_isometry_robustness_scores.append(score)

print(f"Average RipsNet PL isometry robustness score for clean data: {np.mean(clean_ripsnet_pl_isometry_robustness_scores):.6f}")
print(f"Standard deviation RipsNet PL isometry robustness score for clean data: {np.std(clean_ripsnet_pl_isometry_robustness_scores):.6f}")
print(f"Average RipsNet PL isometry robustness score for noisy data: {np.mean(noisy_ripsnet_pl_isometry_robustness_scores):.6f}")
print(f"Standard deviation RipsNet PL isometry robustness score for noisy data: {np.std(noisy_ripsnet_pl_isometry_robustness_scores):.6f}")


# Calculate Isometry Robustness for DistanceMatrixRaggedModel PI Predictions
clean_dm_pi_isometry_robustness_scores = []
for pc in tqdm(clean_data_test, desc='Computing DM PI isometry robustness for clean data'):
    score = compute_isometry_robustness_score(pc, model_dm_pi, n_augment_per_score=5, device=device, seed=42)
    clean_dm_pi_isometry_robustness_scores.append(score)

noisy_dm_pi_isometry_robustness_scores = []
for pc in tqdm(noisy_data_test, desc='Computing DM PI isometry robustness for noisy data'):
    score = compute_isometry_robustness_score(pc, model_dm_pi, n_augment_per_score=5, device=device, seed=42)
    noisy_dm_pi_isometry_robustness_scores.append(score)

print(f"Average DM PI isometry robustness score for clean data: {np.mean(clean_dm_pi_isometry_robustness_scores):.6f}")
print(f"Standard deviation DM PI isometry robustness score for clean data: {np.std(clean_dm_pi_isometry_robustness_scores):.6f}")
print(f"Average DM PI isometry robustness score for noisy data: {np.mean(noisy_dm_pi_isometry_robustness_scores):.6f}")
print(f"Standard deviation DM PI isometry robustness score for noisy data: {np.std(noisy_dm_pi_isometry_robustness_scores):.6f}")

# Calculate Isometry Robustness for DistanceMatrixRaggedModel PL Predictions
clean_dm_pl_isometry_robustness_scores = []
for pc in tqdm(clean_data_test, desc='Computing DM PL isometry robustness for clean data'):
    score = compute_isometry_robustness_score(pc, model_dm_pl, n_augment_per_score=5, device=device, seed=42)
    clean_dm_pl_isometry_robustness_scores.append(score)

noisy_dm_pl_isometry_robustness_scores = []
for pc in tqdm(noisy_data_test, desc='Computing DM PL isometry robustness for noisy data'):
    score = compute_isometry_robustness_score(pc, model_dm_pl, n_augment_per_score=5, device=device, seed=42)
    noisy_dm_pl_isometry_robustness_scores.append(score)

print(f"Average DM PL isometry robustness score for clean data: {np.mean(clean_dm_pl_isometry_robustness_scores):.6f}")
print(f"Standard deviation DM PL isometry robustness score for clean data: {np.std(clean_dm_pl_isometry_robustness_scores):.6f}")
print(f"Average DM PL isometry robustness score for noisy data: {np.mean(noisy_dm_pl_isometry_robustness_scores):.6f}")
print(f"Standard deviation DM PL isometry robustness score for noisy data: {np.std(noisy_dm_pl_isometry_robustness_scores):.6f}")


# Calculate Isometry Robustness for PointNet PI Predictions
clean_pn_pi_isometry_robustness_scores = []
for pc in tqdm(clean_data_test, desc='Computing PointNet PI isometry robustness for clean data'):
    score = compute_isometry_robustness_score(pc, model_PN_PI, n_augment_per_score=5, device=device, seed=42)
    clean_pn_pi_isometry_robustness_scores.append(score)

noisy_pn_pi_isometry_robustness_scores = []
for pc in tqdm(noisy_data_test, desc='Computing PointNet PI isometry robustness for noisy data'):
    score = compute_isometry_robustness_score(pc, model_PN_PI, n_augment_per_score=5, device=device, seed=42)
    noisy_pn_pi_isometry_robustness_scores.append(score)

print(f"Average PointNet PI isometry robustness score for clean data: {np.mean(clean_pn_pi_isometry_robustness_scores):.6f}")
print(f"Standard deviation PointNet PI isometry robustness score for clean data: {np.std(clean_pn_pi_isometry_robustness_scores):.6f}")
print(f"Average PointNet PI isometry robustness score for noisy data: {np.mean(noisy_pn_pi_isometry_robustness_scores):.6f}")
print(f"Standard deviation PointNet PI isometry robustness score for noisy data: {np.std(noisy_pn_pi_isometry_robustness_scores):.6f}")

# Calculate Isometry Robustness for PointNet PL Predictions
clean_pn_pl_isometry_robustness_scores = []
for pc in tqdm(clean_data_test, desc='Computing PointNet PL isometry robustness for clean data'):
    score = compute_isometry_robustness_score(pc, model_PN_PL, n_augment_per_score=5, device=device, seed=42)
    clean_pn_pl_isometry_robustness_scores.append(score)

noisy_pn_pl_isometry_robustness_scores = []
for pc in tqdm(noisy_data_test, desc='Computing PointNet PL isometry robustness for noisy data'):
    score = compute_isometry_robustness_score(pc, model_PN_PL, n_augment_per_score=5, device=device, seed=42)
    noisy_pn_pl_isometry_robustness_scores.append(score)

print(f"Average PointNet PL isometry robustness score for clean data: {np.mean(clean_pn_pl_isometry_robustness_scores):.6f}")
print(f"Standard deviation PointNet PL isometry robustness score for clean data: {np.std(clean_pn_pl_isometry_robustness_scores):.6f}")
print(f"Average PointNet PL isometry robustness score for noisy data: {np.mean(noisy_pn_pl_isometry_robustness_scores):.6f}")
print(f"Standard deviation PointNet PL isometry robustness score for noisy data: {np.std(noisy_pn_pl_isometry_robustness_scores):.6f}")

**Reasoning**:
I need to summarize and compare the performance of `RaggedPersistenceModel`, `DistanceMatrixRaggedModel`, and `PointNet` for predicting Persistence Images and Persistence Landscapes, based on their classification accuracy and isometry robustness scores. I will gather the relevant metrics from the previous code execution and present them in a clear markdown table.



In [ ]:
## Summary of Model Performance

Here's a summary of the performance of the different models (RaggedPersistenceModel/RipsNet, DistanceMatrixRaggedModel, and PointNet) for predicting Persistence Images (PI) and Persistence Landscapes (PL), evaluated using classification accuracy on clean and noisy test sets, and isometry robustness (average and standard deviation of L2 distance between predictions of original and augmented point clouds).

### Persistence Image (PI) - Classification Accuracy (XGBoost)

| Model                  | Clean Test Accuracy | Noisy Test Accuracy |
|------------------------|---------------------|---------------------|
| RipsNet                | 0.636667            | 0.690000            |
| DistanceMatrixRagged   | 0.740000            | 0.776667            |
| PointNet               | 0.933333            | 0.636667            |
| Gudhi (Baseline)       | 1.000000            | 0.666667            |

### Persistence Landscape (PL) - Classification Accuracy (XGBoost)

| Model                  | Clean Test Accuracy | Noisy Test Accuracy |
|------------------------|---------------------|---------------------|
| RipsNet                | 0.703333            | 0.736667            |
| DistanceMatrixRagged   | 0.736667            | 0.743333            |
| PointNet               | 0.883333            | 0.696667            |
| Gudhi (Baseline)       | 0.996667            | 0.363333            |

### Isometry Robustness (Average L2 Distance to Augmented Samples)

| Model (Descriptor)       | Clean Data (Mean ± Std)        | Noisy Data (Mean ± Std)          |
|--------------------------|--------------------------------|----------------------------------|
| RipsNet (PI)             | 0.589158 ± 0.689707            | 0.598095 ± 0.676659              |
| RipsNet (PL)             | 0.040208 ± 0.023749            | 0.039987 ± 0.026292              |
| DistanceMatrixRagged (PI)| 0.000000 ± 0.000000            | 0.000000 ± 0.000000              |
| DistanceMatrixRagged (PL)| 0.000000 ± 0.000000            | 0.000000 ± 0.000000              |
| PointNet (PI)            | 1.348374 ± 0.858867            | 1.398915 ± 1.012282              |
| PointNet (PL)            | 0.223900 ± 0.109701            | 0.245333 ± 0.093165              |

### Comparison and Analysis:

**Persistence Images (PI):**

*   **Classification Accuracy:** Gudhi's raw Persistence Images provide the best accuracy on clean data (1.0), as expected, since it's the ground truth. `PointNet` performs very well on clean data (0.933333), showing its ability to learn PI from point clouds. `DistanceMatrixRagged` has decent accuracy (0.74), while `RipsNet` (using DeepSets) is slightly lower (0.636667). On noisy data, `DistanceMatrixRagged` achieves the highest accuracy (0.776667), indicating its robustness, followed by `RipsNet` (0.69) and then Gudhi's direct computation (0.666667), and `PointNet` (0.636667). This suggests that learning PIs from distance matrices can be beneficial for noisy data.
*   **Isometry Robustness:** Both `DistanceMatrixRagged` models show remarkably low isometry robustness scores (0.000000 mean L2 distance), which suggests perfect invariance to isometries. This is likely due to an internal mechanism that normalizes or is inherently invariant to translations and rotations when generating distance matrices from augmented point clouds. RipsNet (DeepSets) and PointNet show higher average L2 distances, meaning their predictions change more under isometries. RipsNet is more robust than PointNet in this regard.

**Persistence Landscapes (PL):**

*   **Classification Accuracy:** Similar to PIs, Gudhi's raw PLs have very high accuracy on clean data (0.996667), but its performance drops significantly on noisy data (0.363333). `DistanceMatrixRagged` performs best on noisy data (0.743333) and has good performance on clean data (0.736667). `RipsNet` (DeepSets) shows strong performance on noisy data (0.736667) and decent on clean (0.703333). `PointNet` has good accuracy on clean data (0.883333) but drops on noisy data (0.696667).
*   **Isometry Robustness:** Again, `DistanceMatrixRagged` models exhibit perfect isometry invariance (0.000000), which is a key advantage. `RipsNet` (DeepSets) shows very good robustness with low mean L2 distances (0.040208 for clean, 0.039987 for noisy), performing better than `PointNet` (0.223900 for clean, 0.245333 for noisy).

**Overall Observations:**

*   **Robustness to Noise:** The models that learn topological descriptors (`RipsNet`, `DistanceMatrixRagged`, `PointNet`) generally show better classification accuracy on noisy test data compared to directly using Gudhi's computed descriptors. This highlights the learned models' ability to extract more robust topological features.
*   **Isometry Invariance:** The `DistanceMatrixRaggedModel` demonstrates perfect isometry invariance due to its input being the distance matrix, which is inherently invariant to rigid transformations of the point cloud. `RipsNet` (DeepSets) also shows very good robustness to isometries, particularly for PLs. `PointNet` shows the least robustness to isometries, as indicated by its higher average L2 distances.
*   **PointNet vs. DeepSets (RipsNet):** For PIs, PointNet achieved higher clean accuracy but lower noisy accuracy compared to RipsNet. For PLs, PointNet again showed higher clean accuracy but slightly lower noisy accuracy than RipsNet. The `DistanceMatrixRaggedModel` consistently performed well, especially on noisy data, for both PI and PL prediction tasks, and showcased perfect isometry invariance.

This analysis shows that while traditional TDA methods can be highly accurate on clean data, learned approaches, especially those with inherent robustness or specialized inputs like distance matrices, can offer superior performance in the presence of noise and maintain isometry invariance.